In [1]:
import torch 
from math import sqrt 

Vamos a  "simular" una red neuronal completamente conectada con parámetros aleatorios. Digamos que x es la entrada y tenemos batch size de 64. 

In [27]:
x = torch.randn(64, 100) #64 vectores de tamaño 100 

In [3]:
x.mean(), x.std() #std es la desviación estandar

(tensor(0.0145), tensor(0.9855))

Ahora, en cada capa vamos a multiplicar por una matriz y aplicar una función de activación (e.g. relu) 

In [18]:
# una red con 20 capas trono :( 
for i in range(20):
    x @= torch.randn(100, 100)
    torch.relu_(x)

In [19]:
x

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])

In [20]:
x.std()

tensor(nan)

wait! Exploto!

# Qué podemos hacer
Hay varios cosas que se inventaron para tratar amellorar este problema y poder entrenar redes mucho más profundas: 

- 1. Inicializar correctamente las capas
- 2. BatchNorm 
- 3. ResBlocks

In [28]:
# En el caso de Inicializar correctamente las capas
# Para que no truene debemos de multiplicar nuestro 
# vector por la raíz cuadrado de 2/100
for i in range(50): 
    x @= torch.randn(100, 100)*sqrt(2/100) # Vuelve estable
    # pasar por la red neuronal si inicializamos así
    torch.relu_(x)

x.std()

tensor(0.1077)

**Todo esto fue para redes neuronales completamente conectadas y activación de Relu**

*Nota:* Si cambiamos relu por otra cosa, tendrémos que multiplicar por otra cosa para volverlo estable

# Batchnorm 

Esta es una idea muy sencilla. ¿Por qué no normalizar para tener desviación estándar 1 y media 0 después de cada capa? Pues eso es una de las dos cosas que intenta hacer batchnorm: 

x <- x-u/o

Pero ¿cómo? En realidad no conocemos la desviación estándar y la media de x en la capa número 14. Entonces lo que hacemos es usar la desviación estándar y la media en la batch. 

**Nota:** "u" y "o" son vectores con el mismo número de canales que x sin contar la *batch_size*. Es decir, cada canal podría tener diferentes media y desviación. 

Para entonces nunca vas a poder predecir cosas con diferente media o desviación estándar!!

Entonces lo que hace batchnorm es además decide, independientemente, dos parámetros uno multiplicativo (y (gama)) y uno aditivo (beta). Estos parámetros tienen el mismo número de canales que x (obviamente sin contar la batch). 

x <- yx+beta

Casi siempre inicializamos y = (1,1,1;...,1) y beta = (0,0,0,...,0)

En código: 

In [29]:
import torch.nn as nn

In [30]:
B = nn.BatchNorm1d(20) #1d para vectores

In [32]:
x = torch.rand(64, 20)
B(x)

tensor([[-0.0604,  0.1279, -1.0824,  ...,  0.1797, -0.1209, -1.2498],
        [-1.5454,  0.0953, -0.3157,  ..., -0.7441, -1.2516, -0.6801],
        [ 0.9030,  1.1227,  1.5608,  ...,  0.9900,  1.3677,  0.5576],
        ...,
        [ 1.2803,  1.4097, -1.1554,  ..., -1.1472, -0.3524, -0.6955],
        [ 1.3132, -1.2783,  0.2782,  ...,  0.3678,  0.2828,  0.7381],
        [ 1.0917,  0.0105,  0.5509,  ..., -1.3673,  1.3939, -0.5661]],
       grad_fn=<NativeBatchNormBackward>)

In [33]:
B.weight #Estos números son parámetros, van aprendiendo

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], requires_grad=True)

In [34]:
B.bias

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

Batchnorm básicamente quita el overfitting y el desvanecimiento de la gradiente. Por lo tanto usa Batchnorm en todas las redes que hagas Vidale

In [39]:
# Suponiendo que ya estás creando tu red, 
# debería de empezar algo así

nn.Sequential(nn.BatchNorm2d(3), # iniciar con batchnorm
              # para normalizar los datos es una buena 
              # práctica
             nn.Conv2d(3,16,kernel_size=3), 
             nn.ReLU(), 
             nn.BatchNorm2d(16))

Sequential(
  (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (2): ReLU()
  (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

# Resblocks 

Aún así, a las redes neuronales les cuesta trabajo aprender funciones sencillas (como por ejemplo la identidad). 

Entonces se les ocurrió que en vez de que cada "capa" calcule un valor, que más bien cada capa calcule "un error" o "residuo" (por eso residual nets). 

Es decir, que en vez de directamente calcular cuánto debe valer la salida, mejor que calcule cuánto hay que sumarle a la entrada para producir una buena salida: 

x -> f(x) *lo reemplazamos por* x -> x + f(x) 

Donde f podría ser una mini-red neuronal con unas 2 o 3 capas. 

De esta manera, los gradientes pueden "brincar" más sencillamente. 

O sea, en lugar de ir tipo: 

- PuntoA -> intermedio1 -> intermedio2 -> PuntoB

van algo así: 

- PuntoA -> PuntoB

En código:

In [56]:
class ResBlock(nn.Module): 
    def __init__(self, residual):
        super().__init__()
        self.residual = residual 
        
    def forward(self, x):
        return (x + self.residual(x))

¿Cómo lo usamos?

Hay varias maneras de poner la red residual. Aquí hay una posibilidad, pero hay muchas. Experimenta!!

In [57]:
def crear_residual(filters): # Número de filtros de entrada 
    # número de filtros de x básciamente
    bottleneck = (filters+1)//2
    residual = nn.Sequential(
        # Es como una compresión
        nn.Conv2d(filters, bottleneck, kernel_size=1), #Esto es como para reducir el tamaño 
        nn.ReLU(), 
        nn.BatchNorm2d(bottleneck),
        nn.Conv2d(bottleneck, bottleneck, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(bottleneck),
        nn.Conv2d(bottleneck, filters, kernel_size=1),
        nn.ReLU(),
        # Es importante terminar con batchnorm
        nn.BatchNorm2d(filters)
    )
    nn.init.constant_(residual[-1].weight, 0) 
    return (residual)

Además este último batchnorm deberá estar inicializado con 0's (en vez de 1's) para y, como para que empiece siendo la identidad. Esto lo hacemos con nn.init.constant_. Porque quiero que empiece siendo la identidad y a partir de ahí aprenda cosas. 

In [58]:
ResBlock(crear_residual(16))

ResBlock(
  (residual): Sequential(
    (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

Ya tenemos todos los ingredientes de resnet18, resnet34, etcétera!!

In [59]:
import fastai.vision.all as fv

In [60]:
fv.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  